# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [44]:
import re                                                                                     #It is being used.
import numpy as np                                                                            #It is being used.
import pandas as pd                                                                           #It is being used.
from nltk.tokenize import word_tokenize                                                       #It is being used.
from nltk.stem import WordNetLemmatizer                                                       #It is being used.
import nltk                                                                                   #It is being used.
import pickle                                                                                 #It is being used.
from sklearn.model_selection import GridSearchCV                                              #It is being used.
from sklearn.ensemble import RandomForestClassifier                                           #It is being used.
from sklearn.model_selection import train_test_split                                          #It is being used.
from sklearn.pipeline import Pipeline, FeatureUnion                                           #It is being used.
from sklearn.base import BaseEstimator, TransformerMixin                                      #It is being used.
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer                 #It is being used.
from sklearn.metrics import classification_report                                             #It is being used.
from sqlalchemy import create_engine                                                          #It is being used.
from sklearn.multioutput import MultiOutputClassifier                                         #It is being used.
from sklearn.neighbors import KNeighborsClassifier                                            #It is being used.
from sklearn.metrics import  f1_score,precision_score,recall_score,accuracy_score,make_scorer #It is being used.
nltk.download(['punkt','stopwords', 'wordnet', 'averaged_perceptron_tagger'])                 #It is being used. Check again.
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  #It is being used.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
!pip install --upgrade 'sqlalchemy<2.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.21
    Uninstalling SQLAlchemy-2.0.21:
      Successfully uninstalled SQLAlchemy-2.0.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [4]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table('MessageDatabase', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [7]:
#THIS IS THE PIPELINE DESIGN IMPLEMENTED
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x78e543b45360>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
#Predict desired values
y_pred = pipeline.predict(X_test)

#Transform predicted matrices and add column names same with y_test
y_pred_w_names = pd.DataFrame(y_pred, columns = y_test.columns)

#Print each output column and its related scores
for column in y_test.columns:
    print('column_name: ' + column + '\n')
    print(classification_report(y_test[column], y_pred_w_names[column]))

### 6. Improve your model
Use grid search to find better parameters.

In [16]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf2', MultiOutputClassifier(RandomForestClassifier()))
])



# Define hyperparameters to tune
param_grid2 = {
    'clf2__estimator__min_samples_leaf': (1,2),
    #'clf2__estimator__min_samples_split':(2,3,4)
}

In [18]:
clf2 = GridSearchCV(pipeline_2, cv=2, param_grid=param_grid2, verbose=2, n_jobs=-1)
clf2.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x78e543b45360>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf2',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'clf2__estimator__min_samples_leaf': (1, 2)},
             verbose=2)

In [19]:
# Print the best hyperparameters
print("Best Hyperparameters:", clf2.best_params_)

Best Hyperparameters: {'clf2__estimator__min_samples_leaf': 1}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred = clf2.predict(X_test)
y_pred_w_names = pd.DataFrame(y_pred, columns = y_test.columns)

In [21]:
def get_metrics(test_value, predicted_value):
    accuracy = accuracy_score(test_value,predicted_value)
    precision =round(precision_score(test_value,predicted_value,average='micro'))
    recall = recall_score(test_value,predicted_value,average='micro')
    f1 = f1_score(test_value,predicted_value,average='micro')
    return {'Accuracy':accuracy, 'f1 score':f1,'Precision':precision, 'Recall':recall}

In [24]:
train_results = []
for column in y_test.columns:
  result = get_metrics(y_test[column], y_pred_w_names[column])
  train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df


,Accuracy,f1 score,Precision,Recall
0,0.800764,0.800764,1,0.800764
1,0.899312,0.899312,1,0.899312
2,0.995264,0.995264,1,0.995264
3,0.778457,0.778457,1,0.778457
4,0.926509,0.926509,1,0.926509
5,0.952483,0.952483,1,0.952483
6,0.973415,0.973415,1,0.973415
7,0.982277,0.982277,1,0.982277
8,0.966081,0.966081,1,0.966081
9,1.000000,1.000000,1,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [47]:
#Trying to add different classifier
pipeline_3 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
model = pipeline_3.fit(X_train, y_train)

In [ ]:
#Predict desired values
y_pred = model.predict(X_test)

In [ ]:
#Transform predicted matrices and add column names same with y_test
y_pred_w_names = pd.DataFrame(y_pred, columns = y_test.columns)

#Print each output column and its related scores
for column in y_test.columns:
    print('column_name: ' + column + '\n')
    print(classification_report(y_test[column], y_pred_w_names[column]))

### 9. Export your model as a pickle file

In [45]:
pickle.dump(pipeline, open('classifier.pkl', 'wb'))